In [3]:
pip install keras

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
spe=12845//10
vs=4268//10

In [30]:
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(128, 128, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
#classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
#classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

In [31]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2

In [32]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


In [33]:
# Step 2 - Preparing the train/test data and training the model
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               3686528   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)              

In [34]:
# Code copied from - https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('/Users/baigj/Desktop/sign/dataset/train',
                                                 target_size=(128, 128),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('/Users/baigj/Desktop/sign/dataset/test',
                                            target_size=(128 , 128),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 
classifier.fit_generator(
        training_set,
        steps_per_epoch=spe, # No of images in training set
        epochs=10,
        validation_data=test_set,
        validation_steps=vs) # No of images in test set

Found 12845 images belonging to 27 classes.
Found 4268 images belonging to 27 classes.
Epoch 1/10
1284/1284 [==============================] - 148s 116ms/step - loss: 2.1551 - accuracy: 0.3228 - val_loss: 0.5856 - val_accuracy: 0.8031
Epoch 2/10
1284/1284 [==============================] - 157s 122ms/step - loss: 1.0960 - accuracy: 0.6110 - val_loss: 0.2175 - val_accuracy: 0.9230
Epoch 3/10
1284/1284 [==============================] - 177s 138ms/step - loss: 0.7941 - accuracy: 0.7231 - val_loss: 0.1320 - val_accuracy: 0.9697
Epoch 4/10
1284/1284 [==============================] - 182s 141ms/step - loss: 0.6224 - accuracy: 0.7864 - val_loss: 0.0732 - val_accuracy: 0.9817
Epoch 5/10
1284/1284 [==============================] - 176s 137ms/step - loss: 0.5237 - accuracy: 0.8225 - val_loss: 0.0424 - val_accuracy: 0.9946
Epoch 6/10
1284/1284 [==============================] - 169s 131ms/step - loss: 0.4488 - accuracy: 0.8481 - val_loss: 0.0331 - val_accuracy: 0.9955
Epoch 7/10
1284/1284 [===

In [35]:
# Saving the model
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model-bw.h5')
print('Weights saved')

Model Saved
Weights saved
